<a href="https://colab.research.google.com/github/haifeng-jin/Colabs/blob/main/KerasTuner_issue_113.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install keras-tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.4 MB/s 
     |████████████████████████████████| 1.6 MB 39.7 MB/s 


In [ ]:
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import optimizers, Model, Input, regularizers, callbacks
import numpy as np
import keras_tuner

def build_mlp(hp):

    #lays = [inp_dim] + n_nod
    drop_nod=0.0
    main_input = Input(shape=(2,), dtype='float32', name='main_input')


    X = main_input
    for nbr in range(hp.Int(name='layers',min_value=1,max_value=10)):
        X = Dense(units=hp.Int('unit'+str(nbr),5,100,step=10),
                  activation = 'tanh',
                  kernel_regularizer=regularizers.l2(0.0))(X)
        if type(drop_nod) is list:
            X = Dropout(drop_nod[nbr])(X)
        else:
            X = Dropout(drop_nod)(X)

    output = Dense(1, activation = 'sigmoid' )(X)

    method = getattr(optimizers, 'Adam')

    learning_rates = hp.Choice('eta', [0.0001,0.001,0.01,0.025,0.05,0.1,0.5])
    model =  Model(inputs=[main_input], outputs=[output])
    model.compile(optimizer = method(learning_rate = learning_rates),
                  loss = 'binary_crossentropy',
                  metrics=['accuracy'])

    return model

class CustomKerasTuner(keras_tuner.tuners.RandomSearch):
    def run_trial(self, trial, *args, **kwargs):
        hp = trial.hyperparameters
        kwargs['batch_size'] = hp.Int('mini_batch', 2,75,3)
        # kwargs['epochs'] = hp.Int('epochs', 100, 10000, 400)
        kwargs['epochs'] = hp.Int('epochs', 10, 20)
        return super(CustomKerasTuner, self).run_trial(trial, *args, **kwargs)

early_stopping = callbacks.EarlyStopping(monitor ="accuracy",
                                        mode ="max",
                                        patience = 1000,
                                        restore_best_weights = True)


customtuner = CustomKerasTuner(build_mlp, objective=keras_tuner.Objective("accuracy", direction="max"), max_trials=10, overwrite=True)
customtuner.search(np.random.rand(10, 2), np.random.rand(10, 1), validation_split=0.25, callbacks=[early_stopping])
customtuner.results_summary()

Trial 10 Complete [00h 00m 02s]
accuracy: 0.0

Best accuracy So Far: 0.0
Total elapsed time: 00h 00m 21s
Results summary
Results in ./untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
layers: 5
unit0: 75
eta: 0.0001
mini_batch: 2
epochs: 10
unit1: 5
unit2: 5
unit3: 5
unit4: 5
Score: 0.0
Trial summary
Hyperparameters:
layers: 7
unit0: 25
eta: 0.5
mini_batch: 17
epochs: 15
unit1: 35
unit2: 85
unit3: 65
unit4: 45
unit5: 5
unit6: 5
Score: 0.0
Trial summary
Hyperparameters:
layers: 3
unit0: 35
eta: 0.0001
mini_batch: 11
epochs: 10
unit1: 65
unit2: 75
unit3: 75
unit4: 5
unit5: 35
unit6: 15
Score: 0.0
Trial summary
Hyperparameters:
layers: 4
unit0: 45
eta: 0.025
mini_batch: 50
epochs: 19
unit1: 45
unit2: 25
unit3: 85
unit4: 55
unit5: 15
unit6: 25
Score: 0.0
Trial summary
Hyperparameters:
layers: 8
unit0: 15
eta: 0.001
mini_batch: 59
epochs: 19
unit1: 25
unit2: 65
unit3: 95
unit4: 95
unit5: 35
unit6: 35
unit7: 5
Score: 0.0
Trial summary
Hyperparameters:
layers: 3
unit0: 95